# Manipulation.py: Pulling example

This tutorial provides an example of realizing a "pull" motion of a robot arm on a box using of "manipulation.py" and "robot_execution.py". 
In the context of pulling a box with a robot arm, "pulling" refers to the action of applying force to move the box towards or away from a specific direction or location using the robot arm. This typically involves pressing the robot's gripper against the top part of the box and then exerting force to shift the box along a desired path or trajectory. This type of motion is hard to simulate as friction plays a big role in the motion.

As always we start with the necessary imports

In [1]:
import numpy as np
import robotic as ry
import manipulation as manip

Now onto a basic configuration with a red blue box

In [2]:
C = ry.Config()
C.addFile(ry.raiPath('../rai-robotModels/scenarios/pandaSingle.g'))

midpoint = np.array([-0.105, 0.4, 0.705-.025+.1])
C.addFrame("box") \
    .setPosition(midpoint) \
    .setShape(ry.ST.ssBox, size=[0.08, 0.12, 0.08, 0.001]) \
    .setColor([1, 0, 0]) \
    .setContact(1) \
    .setMass(.1)

# For convenience, a few definitions:
gripper = "l_gripper"
box = "box"
table = "table"

C.view()

0

## Defining the pull motion

Now follows the implementation of the pull() function. The initial ManipulationModelling object calculates the start and end time slices, these are the initial contact with the box and the end pose of the robot on the target position. After this we compute the time slices in between with the sub-motion method of the M object. In the first sub-motion we define the path from the starting position of the robot to the pose where it is touching the box at the top. The second sub-motion defines the movement towards the target end pose defined earlier. 

In [3]:
def pull(obj: str, target_pos: list[float], info: str="", vis: bool=False) -> tuple:
	M = manip.ManipulationModelling(C, info, ['l_gripper'])
	M.setup_pick_and_place_waypoints(gripper, obj, 1e-1)
	M.pull([1., 2.], obj, gripper, table)
	M.target_xy_position(2., obj, target_pos)
	M.solve()
	if not M.feasible:
		return False, []

	M1 = M.sub_motion(0, accumulated_collisions=False)
	M1.retractPush([.0, .15], gripper, .03)
	M1.approachPush([.85, 1.], gripper, .03)
	path1 = M1.solve()
	if not M1.feasible:
		return False, []

	M2 = M.sub_motion(1, accumulated_collisions=False)
	path2 = M2.solve()
	if not M2.feasible:
		return False, []

	if vis:
		M1.play(C, 1.)
		C.attach(gripper, obj)
		M2.play(C, 1.)
		C.attach(table, obj)
	
	path = np.append(path1, path2, 0)
	return True, path

## Visualizing and Executing the motion

We will want to test the output of our function with a real robot, or at least a physics simulation. For this porpuse we can define a bot object with which we will be able to interface with the robot.

In [4]:
bot = ry.BotOp(C, useRealRobot=False)
bot.home(C)

Now we try a random "pull" motion attempt_count number of times.

In [5]:
attempt_count = 30
success_count = 0

for l in range(attempt_count):

	target_position = [
		midpoint[0] + np.random.uniform(-.15, .15),
		midpoint[1] + np.random.uniform(-.15, .15),
		0.]
	
	success, path = pull(box, target_position, str(l), vis=True)

	if success:
		bot.move(path, [3])
		while bot.getTimeToEnd() > 0.:
			bot.sync(C)
		
	success_count += 1 if success else 0
	
print(f"Successful motions: {success_count}/{attempt_count}")

  -- feasible:0
     { time: 0.043493, evals: 140, done: 1, feasible: 1, sos: 0.0551733, f: 0, ineq: 0, eq: 0.0920524 }
  -- feasible:sub_motion_0--0
     { time: 0.026816, evals: 34, done: 1, feasible: 1, sos: 0.459208, f: 0, ineq: 6.01906e-08, eq: 0.000153713 }
  -- feasible:sub_motion_1--0
     { time: 0.004879, evals: 8, done: 1, feasible: 1, sos: 0.0658799, f: 0, ineq: 0, eq: 3.90804e-05 }
  -- feasible:1
     { time: 0.020329, evals: 67, done: 1, feasible: 1, sos: 0.0335756, f: 0, ineq: 0, eq: 0.169063 }
  -- feasible:sub_motion_0--1
     { time: 0.043263, evals: 52, done: 1, feasible: 1, sos: 0.288627, f: 0, ineq: 0.000613689, eq: 0.000278802 }
  -- feasible:sub_motion_1--1
     { time: 0.00312, evals: 7, done: 1, feasible: 1, sos: 0.00712288, f: 0, ineq: 0, eq: 4.26719e-06 }
  -- feasible:2
     { time: 0.041599, evals: 118, done: 1, feasible: 1, sos: 0.0281162, f: 0, ineq: 0, eq: 0.102084 }
  -- feasible:sub_motion_0--2
     { time: 0.016104, evals: 22, done: 1, feasible: 1, s

KeyboardInterrupt: 